In [1]:
import pandas as pd
import numpy as np
import random
import copy

# Create directed graph from dataset
graph = {}
with open('./datasets/facebook_dataset.txt', 'r') as f:
    for line in f:
        k, v = line.split()
        if k not in graph:
            graph[k] = []
        graph[k].append(v)

        if v not in graph:
            graph[v] = []
        graph[v].append(k)

In [3]:
# Construct Reverse Set of Graph G for estimation of influence function
def transpose_graph(graph):
    transposed_graph = {node: [] for node in graph}
    for node in graph:
        for neighbor in graph[node]:
            transposed_graph[neighbor].append(node)

    return transposed_graph

In [4]:
# Building hypergraph with Independent cascade process
def independent_cascade(graph, R, probabilities, topic):
    V = set(graph.keys())  # Set of nodes in G
    H = [{} for _ in range(topic)]  # Set of hyperedges in the hypergraph
    for k in range(topic):    
        for _ in range(R):
            seed_node = random.choice(list(V))  # Choose a node uniformly at random from G
            if seed_node not in H[k]:
                H[k][seed_node] = set()
            activated_nodes = set()             # Set of nodes discovered during influence spread
            activated_nodes.add(seed_node)
            newly_activated_nodes = set(seed_node)
            newly_activated_nodes.add(seed_node)
            while newly_activated_nodes:
                newly_activated_nodes_next = set()
                for node in newly_activated_nodes:
                    if node in graph:
                        for neighbor in graph[node]:
                            if neighbor not in activated_nodes:
                                if random.random() <= probabilities[node][neighbor][k]:
                                    newly_activated_nodes_next.add(neighbor)
                                    activated_nodes.add(neighbor)
                newly_activated_nodes = newly_activated_nodes_next
            H[k][seed_node].update(activated_nodes)
    return H

In [5]:
# Initilalize probabilities
def initialize_probabilities(graph, topics):
    probabilities = {}
    for k,v in graph.items():
        if k not in probabilities:
            probabilities[k] = {}
        for e in v:
            probabilities[k][e] = []
    for u in graph:
        for v in graph[u]:
            dv = len(graph[u])  # In-degree of v
            probability = [2*i/(topics*dv) for i in range(1, topics+1)]
            random.shuffle(probability)
            probabilities[u][v] = probability
    return probabilities

In [6]:
# Initialize Parameters
topic = 1
reverse_set = transpose_graph(graph)
probs = initialize_probabilities(reverse_set,topic)

In [7]:
# Initilize estimated Graph
import math
R = int(len(reverse_set.keys())* topic**2*math.log(len(reverse_set.keys()))) # O(n*k**2*log(n)*log(k))
print(f'Theoritical Bound for Number of Simulations: {R}')
estimate = independent_cascade(reverse_set, R, probs, topic)

Theoritical Bound for Number of Simulations: 33538


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv

# Define the Graph Attention Network model
class GATLinkPrediction(nn.Module):
    def __init__(self, num_nodes, embed_dim, num_heads):
        super(GATLinkPrediction, self).__init__()
        self.embedding = nn.Embedding(num_nodes, embed_dim)
        self.conv1 = GATConv(embed_dim, embed_dim, heads=num_heads, dropout=0.6)
        self.conv2 = GATConv(embed_dim * num_heads, 1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.embedding(x)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)

In [ ]:
# Create a mapping of nodes to indices
node_indices = {node: i for i, node in enumerate(estimate.keys())}

# Convert the dataset into a PyTorch Geometric Data object
edges = [(node_indices[source], node_indices[target]) for source, targets in estimate.items() for target in targets]
edges = torch.tensor(edges, dtype=torch.long).t().contiguous()
x = torch.arange(len(node_indices))
data = Data(x=x, edge_index=edges)

# Define the model
num_nodes = len(node_indices)
embed_dim = 64
num_heads = 4
model = GATLinkPrediction(num_nodes, embed_dim, num_heads)

In [ ]:
# Set device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
# Training loop
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output[data.train_mask].view(-1), data.y[data.train_mask].view(-1))
    loss.backward()
    optimizer.step()
    return loss.item()

# Testing loop
def test(model, data):
    model.eval()
    with torch.no_grad():
        output = model(data)
        return output

In [ ]:
# Prepare training, validation, and testing masks
num_training_samples = int(0.8 * num_nodes)
num_validation_samples = int(0.1 * num_nodes)
train_mask = torch.zeros(num_nodes, dtype=torch.bool)
train_mask[:num_training_samples] = 1
data.train_mask = train_mask
validation_mask = torch.zeros(num_nodes, dtype=torch.bool)
validation_mask[num_training_samples:num_training_samples+num_validation_samples] = 1
data.val_mask = validation_mask

# Split the data into training, validation, and testing sets
data = data.to(device)
data.y = torch.zeros(num_nodes, dtype=torch.float).to(device)
data.y[:num_training_samples] = 1.0

In [11]:
# Define lists to store train and validation losses
train_losses = []
validation_losses = []

epochs = 100
# Train the model
for epoch in range(epochs):
    train_loss = train(model, data, optimizer, criterion)
    train_losses.append(train_loss)

    output = test(model, data)
    validation_loss = criterion(output[data.val_mask].view(-1), data.y[data.val_mask].view(-1)).item()
    validation_losses.append(validation_loss)

    print(f"Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Validation Loss: {validation_loss:.4f}")


OutOfMemoryError: CUDA out of memory. Tried to allocate 8.65 GiB (GPU 0; 12.00 GiB total capacity; 9.50 GiB already allocated; 674.89 MiB free; 9.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import matplotlib.pyplot as plt
# Plot train and validation losses
plt.plot(train_losses, label='Train Loss')
plt.plot(validation_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()